## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Описание данных:

    children — количество детей в семье
    days_employed — общий трудовой стаж в днях
    dob_years — возраст клиента в годах
    education — уровень образования клиента
    education_id — идентификатор уровня образования
    family_status — семейное положение
    family_status_id — идентификатор семейного положения
    gender — пол клиента
    income_type — тип занятости
    debt — имел ли задолженность по возврату кредитов
    total_income — ежемесячный доход
    purpose — цель получения кредита

Я планирую изучить и предобработать входные данные, затем, уже на очищенных данных, построить сводные таблицы, чтобы ответить на поставленный вопрос.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
from collections import Counter
from pymystem3 import Mystem
from nltk.stem import SnowballStemmer 

In [2]:
m = Mystem()
russian_stemmer = SnowballStemmer('russian')

In [3]:
clients = pd.read_csv('/datasets/data.csv')

#### Общая информция о данных

In [4]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Можно заметить, что столбцы total_income и days_employed содержат пропущенные значения. 

In [5]:
clients.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Мы имеем таблицу с 21525 примерами и 12 признаками. Столбцы имеют пустые, отрицательные, логически слишком большие (в столбце стаж) значения, значения с разным регистром, числа имеют большое количество цифр после точки. На мой взгляд причина пропусков в столбце стаж и доход - отсутствие справки о доходах и трудовой. Либо их решили донести позже, либо доход и занятость не официальный и подтвердить документами их не могут. Также, возможно, кто-то из них является безработным, проверим это. Наверняка есть еще ошибки в данных, в процессе работы рассмотрим каждый столбец в отдельности.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [6]:
# Посчитаем количество пропущенных значений
clients.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [7]:
# Посмортим на данные, где значения пропущены
clients_with_null = clients[clients['days_employed'].isnull() & clients['total_income'].isnull()]

In [8]:
clients_with_null.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


In [9]:
clients_with_null.shape

(2174, 12)

In [10]:
# Посмотрим у людей с каким типом занятости не указан доход и стаж
clients_with_null['income_type'].unique()

array(['пенсионер', 'госслужащий', 'компаньон', 'сотрудник',
       'предприниматель'], dtype=object)

Все эти категории имеют доход - заработная плата или пенсия.

In [11]:
# Среди них не оказалось безработных, посмотрим какие значения у них.
clients[clients['income_type'] == 'безработный'].head(30)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3133,1,337524.466835,31,среднее,1,женат / замужем,0,M,безработный,1,59956.991984,покупка жилья для сдачи
14798,0,395302.838654,45,Высшее,0,гражданский брак,1,F,безработный,0,202722.511368,ремонт жилью


Такие пропущенные значения можно заменить либо нулями, либо медианой, либо средним. Удалить их нельзя, т.к. это 10 % данных.
На мой взгляд причина этих пропусков - отсутствие справки о доходах.
Попробуем заменить их медианами зарплат по каждому типу занятости, т.к. она снижает влияние выбросов на рассматриваемый признак.

In [12]:
# Посмотрим сколько примеров с ненулевым доходом по каждому типу занятости
clients.groupby('income_type')['total_income'].count()

income_type
безработный            2
в декрете              1
госслужащий         1312
компаньон           4577
пенсионер           3443
предприниматель        1
сотрудник          10014
студент                1
Name: total_income, dtype: int64

Предприниматель всего один, поэтому медиана по группе 'Предприниматель', по моему мнению, не релеванта. Предпринимателя, который не имеет дохода, удалим.

In [13]:
clients[(clients['income_type'] == 'предприниматель')]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
5936,0,NaN,58,высшее,0,женат / замужем,0,M,предприниматель,0,NaN,покупка жилой недвижимости
18697,0,-520.848083,27,высшее,0,гражданский брак,1,F,предприниматель,0,499163.144947,на проведение свадьбы


In [14]:
clients.drop([5936], inplace = True)

In [15]:
clients.isnull().sum()

children               0
days_employed       2173
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2173
purpose                0
dtype: int64

In [16]:
clients['total_income'].describe()

count    1.935100e+04
mean     1.674223e+05
std      1.029716e+05
min      2.066726e+04
25%      1.030532e+05
50%      1.450179e+05
75%      2.034351e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [17]:
clients.groupby('income_type')['total_income'].median()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

In [18]:
# Заменим пропуски на медиану.
clients_with_null = clients[clients['days_employed'].isnull() & clients['total_income'].isnull()]
income_types = clients_with_null['income_type'].unique()
for income_type in income_types:
    income_median_by_type = clients[clients['income_type'] == income_type]['total_income'].median()
    current_clients = clients[(clients['income_type'] == income_type) & (clients['total_income'].isnull())]
    clients.loc[current_clients.index, 'total_income'] = current_clients['total_income'].fillna(income_median_by_type)

In [19]:
# Проверка
clients.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       19351 non-null float64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null float64
purpose             21524 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.8+ MB


In [20]:
# Проверка
clients[clients['days_employed'].isnull()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,118514.486412,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,150447.935283,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,118514.486412,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,150447.935283,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,118514.486412,сыграть свадьбу


Значения в total_income заменились на медиану

Теперь займёмся столбцом days_employed

In [21]:
# Проверим, что нет значений равных нулю
clients[clients['days_employed'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [22]:
# Пока нам не нужны значения столбца days_employed, поэтому пропущенные заменим на 0
clients['days_employed'] = clients['days_employed'].fillna(0)

In [23]:
# Проверка
clients.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Обнаруженные пропущенные значения были заменены. В столбце total_income - медианами зарплат по каждому типу занятости, в days_employed на нули.

### Замена типа данных

In [24]:
# Сколько примеров имеют отрицательный стаж
clients[(clients['days_employed'] < 0 )].shape

(15906, 12)

In [25]:
clients['days_employed'].describe()

count     21524.000000
mean      56681.507910
std      134873.342873
min      -18388.949901
25%       -2518.268286
50%        -982.625876
75%           0.000000
max      401755.400475
Name: days_employed, dtype: float64

Столбец days_employed имеет отрицательные float значения. Изменим тип данных на int и сделаем значения положительными.

In [26]:
clients['days_employed'] = clients['days_employed'].apply(abs)

In [27]:
# Проверка
clients[(clients['days_employed'] < 0 )].shape

(0, 12)

In [28]:
clients.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [29]:
clients['days_employed'] = clients['days_employed'].astype('int')

In [30]:
clients['days_employed'].describe()

count     21524.000000
mean      60158.764960
std      133358.373696
min           0.000000
25%         610.000000
50%        1808.000000
75%        4779.000000
max      401755.000000
Name: days_employed, dtype: float64

In [31]:
# Изменим также тип данных столбца total_income
clients['total_income'] = clients['total_income'].astype('int')

In [32]:
# Проверка. Все числовые значения теперь имеют тип int.  
clients.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       21524 non-null int64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null int64
purpose             21524 non-null object
dtypes: int64(7), object(5)
memory usage: 2.8+ MB


In [33]:
clients.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [34]:
# Рассмотрим значения столбца с возрастом
clients['dob_years'].describe()

count    21524.000000
mean        43.292697
std         12.574476
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

### Вывод

Были изменен тип данных столбцов days_employed, dob_years и total_income с типа float на тип int, таким  образом я избавилась от большого количества цифр после точки. Отрицательные значения стажа переведены в положительные. Остались слишком большие значения в стаже (401755 дней - это 1100 лет).

### Обработка дубликатов

In [36]:
# Количество дубликатов 
clients.duplicated().sum()

54

In [37]:
# Удалим дубликаты
clients = clients.drop_duplicates().reset_index(drop=True)

In [38]:
clients.duplicated().sum()

0

In [39]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21470 entries, 0 to 21469
Data columns (total 12 columns):
children            21470 non-null int64
days_employed       21470 non-null int64
dob_years           21470 non-null int64
education           21470 non-null object
education_id        21470 non-null int64
family_status       21470 non-null object
family_status_id    21470 non-null int64
gender              21470 non-null object
income_type         21470 non-null object
debt                21470 non-null int64
total_income        21470 non-null int64
purpose             21470 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Было найдено и удалено 54 дубликата

### Лемматизация

In [40]:
# Каким образом были указаны формы образования
clients['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Очевидно, что мы имеем одинаковые уровни образования, но напечатанные в разном регистре.

In [41]:
clients['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Многие цели кредита по смыслу также совпадают, но написаны по-разному. Т.е. мы можем выделить несколько категорий целей.

In [42]:
# Приведем образование к нижнему регистру
clients['education'] = clients['education'].str.lower()

In [43]:
# Найдем леммы в столбце education
clients['education'] = clients['education'].apply(m.lemmatize)

In [44]:
clients.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,"[высокий, \n]",0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,"[средний, \n]",1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,"[средний, \n]",1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,"[средний, \n]",1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,"[средний, \n]",1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [45]:
def lemma_split(row):
    return ''.join(row[:-1])

In [46]:
clients['education'] = clients['education'].apply(lemma_split)

In [47]:
clients.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высокий,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,средний,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,средний,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,средний,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,средний,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [48]:
clients['education'].unique()

array(['высокий', 'средний', 'неоконченный высокий', 'начальный',
       'ученый степень'], dtype=object)

In [49]:
# Найдем леммы в столбце purpose
clients['purpose'] = clients['purpose'].apply(m.lemmatize)

In [50]:
clients.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высокий,0,женат / замужем,0,F,сотрудник,0,253875,"[покупка, , жилье, \n]"
1,1,4024,36,средний,1,женат / замужем,0,F,сотрудник,0,112080,"[приобретение, , автомобиль, \n]"
2,0,5623,33,средний,1,женат / замужем,0,M,сотрудник,0,145885,"[покупка, , жилье, \n]"
3,3,4124,32,средний,1,женат / замужем,0,M,сотрудник,0,267628,"[дополнительный, , образование, \n]"
4,0,340266,53,средний,1,гражданский брак,1,F,пенсионер,0,158616,"[сыграть, , свадьба, \n]"


In [51]:
clients['purpose'] = clients['purpose'].apply(lemma_split)

In [52]:
clients['purpose'].unique()

array(['покупка жилье', 'приобретение автомобиль',
       'дополнительный образование', 'сыграть свадьба',
       'операция с жилье', 'образование', 'на проведение свадьба',
       'покупка жилье для семья', 'покупка недвижимость',
       'покупка коммерческий недвижимость', 'покупка жилой недвижимость',
       'строительство собственный недвижимость', 'недвижимость',
       'строительство недвижимость', 'на покупка подержать автомобиль',
       'на покупка свой автомобиль',
       'операция с коммерческий недвижимость',
       'строительство жилой недвижимость', 'жилье',
       'операция со свой недвижимость', 'автомобиль',
       'заниматься образование', 'сделка с подержанный автомобиль',
       'получение образование', 'свадьба',
       'получение дополнительный образование', 'покупка свой жилье',
       'операция с недвижимость', 'получение высокий образование',
       'свой автомобиль', 'сделка с автомобиль', 'профильный образование',
       'высокий образование', 'покупка жилье 

Создадим категории кредитов: недвижимость = жилье, образование, автомобиль, свадьба.


In [53]:
def new_categories(row):
    if 'жилье' in row or 'недвижимость' in row:
        row = 'недвижимость'
    elif 'образование' in row:
        row = 'образование'
    elif 'автомобиль' in row:
        row = 'автомобиль'
    elif 'свадьба' in row:
        row = 'свадьба'
    return row

In [54]:
clients['purpose'] = clients['purpose'].apply(new_categories)

In [55]:
clients['purpose'].unique()

array(['недвижимость', 'автомобиль', 'образование', 'свадьба'],
      dtype=object)

In [56]:
clients.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высокий,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,4024,36,средний,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,5623,33,средний,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,4124,32,средний,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,340266,53,средний,1,гражданский брак,1,F,пенсионер,0,158616,свадьба


### Вывод

Были найдены проблемы с регистром в образовании, а также одинаковые цели, но написанные по-разному. Образование приведено к 5 категориям в нижнем регистре - высокий, средний, неоконченный высокий, начальный, ученый степень.  Цели к 4 категориям - недвижимость, автомобиль, образование, свадьба.

### Логические ошибки

In [57]:
clients.groupby('children')['children'].count()

children
-1        47
 0     14106
 1      4809
 2      2052
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

In [58]:
clients[clients['children'] == -1].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,4417,46,средний,1,гражданский брак,1,F,сотрудник,0,102816,образование
705,-1,902,50,средний,1,женат / замужем,0,F,госслужащий,0,137882,автомобиль
742,-1,3174,57,средний,1,женат / замужем,0,F,сотрудник,0,64268,образование
800,-1,349987,54,средний,1,Не женат / не замужем,4,F,пенсионер,0,86293,образование
941,-1,0,57,средний,1,женат / замужем,0,F,пенсионер,0,118514,автомобиль


Проблема: 47 человек имеют -1 ребенка и 76 имеют 20 детей.
На мой взгляд это может быть опечаткой и имелось в виду, что имеется один ребенок вместо -1 и 2 вместо 20. 

In [59]:
# Сделаем всех детей положительными
clients['children'] = clients['children'].apply(abs)

In [60]:
clients[clients['children'] == -1]['children'].count()

0

In [61]:
# Заменим 20 детей на 2
clients['children'] = clients['children'].replace(20, 2)

In [62]:
clients[clients['children'] == 20]['children'].count()

0

In [63]:
clients.groupby('children')['children'].count()

children
0    14106
1     4856
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

In [64]:
clients['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [65]:
clients[clients['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10689,0,2358,24,неоконченный высокий,2,гражданский брак,1,XNA,компаньон,0,203905,недвижимость


In [66]:
# одно значение с неопределенным полом ни на что не влияет, поэтому удалим его
clients = clients[clients['gender'] != 'XNA']

### Вывод

Произведено устранение опечаток: отрицательное количество детей заменено положительным, 20 детей заменены на 2. Удалена строка с неопределенным полом.

### Категоризация данных

Создадим столбец с категорями семей по количеству детей.

In [67]:
def categorize_family(row):
    if row['children'] == 0:
        categorie = 'не имеют детей'
    elif row['children'] == 1:
        categorie = 'имеют 1 ребенка'
    elif  row['children'] == 2:
        categorie = 'имеют 2-х детей'
    elif row['children'] >= 3:
        categorie = 'многодетные'
    return categorie

In [68]:
clients['family_categorie'] = clients.apply(categorize_family, axis=1)

In [69]:
clients.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,family_categorie
0,1,8437,42,высокий,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,имеют 1 ребенка
1,1,4024,36,средний,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,имеют 1 ребенка
2,0,5623,33,средний,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,не имеют детей
3,3,4124,32,средний,1,женат / замужем,0,M,сотрудник,0,267628,образование,многодетные
4,0,340266,53,средний,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,не имеют детей


In [70]:
# Рассмотрим столбец total_income и создадим для него категории по уровню дохода
clients['total_income'].describe()

count    2.146900e+04
mean     1.652780e+05
std      9.813122e+04
min      2.066700e+04
25%      1.076500e+05
50%      1.425940e+05
75%      1.957510e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [71]:
clients['total_income'].min()

20667

In [72]:
clients['total_income'].mean()

165278.02971726676

In [73]:
clients['total_income'].median()

142594.0

In [74]:
clients['total_income'].max()

2265604

В данном случае мне не понравилось как выглядит вывод команды describe, поэтому я вывела интересующие меня параметры отдельно.

In [75]:
clients['categorie_income'] = pd.qcut(clients['total_income'], 4, labels=[
    'нижний средний класс', 'средний класс', 'верхний средний класс', 'высокодоходные'])

In [76]:
clients.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,family_categorie,categorie_income
0,1,8437,42,высокий,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,имеют 1 ребенка,высокодоходные
1,1,4024,36,средний,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,имеют 1 ребенка,средний класс
2,0,5623,33,средний,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,не имеют детей,верхний средний класс
3,3,4124,32,средний,1,женат / замужем,0,M,сотрудник,0,267628,образование,многодетные,высокодоходные
4,0,340266,53,средний,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,не имеют детей,верхний средний класс


### Вывод

Клиенты были разделены на три группы по количеству детей: не имеют детей; имеют 1 ребенка, имеют 2 детей; многодетные. Также были введены четыре категории дохода в соответствии с квантилями доходов - 'нижний средний класс', 'средний класс', 'верхний средний класс', 'высокодоходные'. При использовании этого подхода группы по категориям дохода имеют одинаковые размеры. Созданы два столбца с этими данными.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [77]:
# Здесь и далее 
# столбец count - количество примеров данной категории, 
# столбец sum - количество примеров данной категории, которые имели  задолженность по возврату кредитов, 
# столбец mean - отношение проблемных заемщиков данной категории ко всем заемщикам данной категории. 
clients.groupby('family_categorie')['debt'].agg(['count', 'sum', 'mean'])

,count,sum,mean
family_categorie,,,
имеют 1 ребенка,4856,445,0.091639
имеют 2-х детей,2128,202,0.094925
многодетные,380,31,0.081579
не имеют детей,14105,1063,0.075363


### Вывод

Те, кто не имеют детей имеют меньше проблем с возвратом кредита, но всего лишь на 1%. Мое мнение, количество детей не серьезно влияет на возврат кредита.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [78]:
clients.groupby('family_status')['debt'].agg(['count', 'sum', 'mean'])

,count,sum,mean
family_status,,,
Не женат / не замужем,2810,274,0.097509
в разводе,1195,85,0.071130
вдовец / вдова,959,63,0.065693
гражданский брак,4162,388,0.093224
женат / замужем,12343,931,0.075427


Интересно, какая часть от всех клиентов имеет проблемы с возвратом кредита

In [79]:
clients['debt'].agg(['mean'])

mean    0.081094
Name: debt, dtype: float64

### Вывод

Люди в гражданском браке и не замужние/ не женатые возвращают кредиты хуже всего, т.к. их отношения не закреплены законом и, возможно, поэтому они могут иметь менее стабильное финансовое положение и испытывают трудности с выплатой кредита.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [80]:
clients.groupby('categorie_income')['debt'].agg(['count', 'sum', 'mean']).sort_values('mean')

,count,sum,mean
categorie_income,,,
высокодоходные,5367,383,0.071362
нижний средний класс,5368,427,0.079545
верхний средний класс,5244,448,0.085431
средний класс,5490,483,0.087978


### Вывод

Люди, относящиеся к категории "высокодоходные", имеют меньше всего проблем с возвратом, т.к. имеют благополучное финансовое положение.
Люди, имеющие низкий доход, также отличаются показателем возврата выше среднего(0.081), это странно, но можно попробовать объяснить тем, что у них нет средств оплачивать штрафы за просрочку платежа, это заставляет их следовать соглашению с банком.

- Как разные цели кредита влияют на его возврат в срок?

In [81]:
clients.pivot_table(index=['purpose'], values=['debt'], aggfunc='mean').sort_values('debt')

,debt
purpose,
недвижимость,0.072327
свадьба,0.079657
образование,0.092177
автомобиль,0.093547


### Вывод

Доля возврата кредитов без проблем выше всего у заемщиков на недвижимость. В категории "недвижимость" присутствуют клиенты, которые взяли кредит для покупки коммерческой или жилой недвижимости(10 тысяч записей), и только 612 клиентов, которые взяли кредит на ремонт. Получается, что при оформлении кредита на недвижимость, чаще всего она является собственностью банка, пока не будет выплачен кредит. В противном случае человек может остаться без жилья или без недвижимости, которая приносит доход.
Кредиты на свадьбу берут люди, которые узаконивают свои отношения и стабилизируют финансовое положение, а это, как мы заметили в одном из предыдущих выводов, сильно влияет на возврат кредитов. 

Кредиты на автомобили возвращаются хуже всего, можно предположить, что это связано с тем, что люди беспокоятся о мнении окружающих и покупают себе машину, которую не могут себе позволить. К тому же в автосалонах часто навязывают дорогостоящие услуги и более дорогие комплектации, которые в дальнешем клиент оплатить не в состоянии.

### Шаг 4. Общий вывод

In [82]:
# Посмотрим сводную таблицу 
clients.pivot_table(index=['family_status', 'purpose', 'categorie_income'],
                    values=['debt'], aggfunc='mean').sort_values('debt').head(20)

debt
family_status         purpose      categorie_income               
вдовец / вдова        недвижимость нижний средний класс   0.017241
в разводе             образование  средний класс          0.032258
гражданский брак      свадьба      высокодоходные         0.050347
в разводе             недвижимость верхний средний класс  0.050847
                      автомобиль   высокодоходные         0.052632
                      недвижимость высокодоходные         0.054645
вдовец / вдова        образование  высокодоходные         0.060606
в разводе             недвижимость нижний средний класс   0.061728
                      автомобиль   средний класс          0.061728
вдовец / вдова        автомобиль   высокодоходные         0.062500
женат / замужем       недвижимость высокодоходные         0.063641
вдовец / вдова        недвижимость средний класс          0.064516
гражданский брак      недвижимость нижний средний класс   0.065217
Не женат / не замужем недвижимость средний класс          0.065728
в разводе             образование  верхний средний класс  0.066667
вдовец / вдова        образование  верхний средний класс  0.066667
                                   средний класс          0.066667
женат / замужем       недвижимость верхний средний класс  0.067669
вдовец / вдова        недвижимость высокодоходные         0.068627
женат / замужем       образование  высокодоходные         0.069692

In [83]:
clients.pivot_table(index=['family_status', 'purpose', 'categorie_income'],
                    values=['debt'], aggfunc='mean').sort_values('debt').tail(20)

debt
family_status         purpose      categorie_income               
вдовец / вдова        образование  нижний средний класс   0.098361
Не женат / не замужем образование  нижний средний класс   0.099338
                      недвижимость верхний средний класс  0.099744
в разводе             автомобиль   верхний средний класс  0.100000
                      образование  высокодоходные         0.104167
гражданский брак      автомобиль   средний класс          0.107438
                      недвижимость средний класс          0.109312
                      образование  высокодоходные         0.111111
                      недвижимость верхний средний класс  0.112840
в разводе             недвижимость средний класс          0.116883
Не женат / не замужем автомобиль   высокодоходные         0.120000
гражданский брак      автомобиль   нижний средний класс   0.121495
                      образование  нижний средний класс   0.135922
Не женат / не замужем автомобиль   средний класс          0.141361
                      образование  средний класс          0.143939
вдовец / вдова        автомобиль   верхний средний класс  0.146341
Не женат / не замужем автомобиль   верхний средний класс  0.158940
гражданский брак      образование  верхний средний класс  0.159574
                      автомобиль   верхний средний класс  0.161290
                      образование  средний класс          0.185185

In [84]:
# Основной вопрос работы - влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок
clients.pivot_table(index=['family_status', 'family_categorie'],
                    values=['debt'], aggfunc=['mean', 'count']).sort_values(('mean', 'debt'))

mean count
                                            debt  debt
family_status         family_categorie                
вдовец / вдова        многодетные       0.000000     7
                      не имеют детей    0.062574   847
в разводе             имеют 1 ребенка   0.066456   316
женат / замужем       не имеют детей    0.069058  7472
в разводе             не имеют детей    0.070153   784
женат / замужем       многодетные       0.070175   285
                      имеют 1 ребенка   0.082224  3004
в разводе             многодетные       0.083333    12
гражданский брак      не имеют детей    0.083577  2740
вдовец / вдова        имеют 1 ребенка   0.086420    81
Не женат / не замужем не имеют детей    0.092838  2262
гражданский брак      имеют 2-х детей   0.092958   355
женат / замужем       имеют 2-х детей   0.093552  1582
в разводе             имеют 2-х детей   0.096386    83
Не женат / не замужем имеют 1 ребенка   0.114537   454
гражданский брак      имеют 1 ребенка   0.117882  1001
Не женат / не замужем имеют 2-х детей   0.119048    84
гражданский брак      многодетные       0.121212    66
вдовец / вдова        имеют 2-х детей   0.125000    24
Не женат / не замужем многодетные       0.200000    10

После изучения данных были найдены и устранены:
*  пропуски
*  отрицательные зачения в столбце стаж и количество детей
*  дубликаты

Параметр образование был преведен к нижнему регистру, в нем и в параметре цель кредита найдены леммы, на основе полученных значений были введены новые категории для этих параметров.

Клиенты были разделены на категории, исходя из количества детей и квантилей дохода(4 равные части выборки).

Отвечая на вопрос заказчика - влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок - да, влияет. Лучше всего отдают кредиты клиенты, которые были или сейчас в официальном браке и не имеют детей. Помимо этого было установлено, что также на погашение кредита влияет цель кредита и доход клиента.

Идеальный портрет заемщика - вдовец/вдова без детей с доходом ниже среднего, который/ая берет кредит на недвижимость.
Хуже всего отдают кредиты - клиенты с детьми, не в официальном браке, со средним и выше среднего доходом, которые берут кредиты на автомобиль или образование.